In [ ]:
# st = '(+ 1  2)'
# st = '(+ 1 2 (sqrt 4))'
# st = '(+ 1 (sqrt 4))'
st = '(if ( < x 2) 42 0)'
# st = '(lambda (x y) (+ 1 2))' # lambda no cheme
# eval_scheme(st)

In [ ]:
import math
import operator as op
from collections import deque, ChainMap

Symbol = str

default_context = {
#     '+': lambda x, y: x + y,
#     '-': lambda x, y: x - y,
#     '*': lambda x, y: x * y,
#     '/': lambda x, y: x / y,
    '+': op.add,
    '-': op.sub,
    '*': op.mul,
    '/': op.truediv,
    '<': op.lt,
    '<=': op.le,
    'if': lambda cond, true, false: true if cond else false,
    'lambda': ...,
    'sqrt': math.sqrt,
}

# parse = analise sintatica. Pegar uma string e converter em uma estrutura util.
def eval_scheme(st, ctx=default_context):
    # AST: Abstract Syntax Tree
    ast = parse(st) 
    return run_ast(ast)

def parse(str):
    tokens = lex(st)
    return parse_tokens(deque(tokens))

def lex(st):
    return st.replace('(', ' ( ').replace(')', ' ) ').split()
    #"( ) ( 41234 2134 23 sqrt foo-bar + - "

def parse_tokens(tokens):
    if tokens[0] == '(':
        del tokens[0]
        result = []
        
        while tokens[0] != ')':
            result.append(parse_tokens(tokens))
        del tokens[0]
        return result
    try:
        result = float(tokens[0])
        del tokens[0]
        return result
    
    except ValueError:
        return deque(tokens).popleft()

In [ ]:
lex(st)

In [ ]:
parse_tokens(lex(st))

In [ ]:
eval('x + 2', {'x': 40})

In [ ]:
def run_ast(ast, ctx):
    if isinstance(ast, (float, int)):
        return ast
    elif isinstance(ast, Symbol):
        try:
            return ctx[ast]
        except KeyError:
            raise NameError(f'unknow variable: {ast}') #format string. string interpoladora
            
    head, *args = ast
    if head == 'lambda':
        arg_names, body = args
        
        def function(*args_values):
            local = dict(zip(arg_names, arg_values))
            new_ctx = ChainMap(local, ctx)
            return run_ast(body, new_ctx)
        return function
            
    else:
        func = run_ast(head, ctx)
        args = [run_ast(arg, ctx) for arg in args]
        return func(*args)
        
        
f = run_ast(parse(st), {'x': 10, **default_context})
f(40, 2)

## Python tricks examples below

In [ ]:
lst = [1,2,3,4,5,6]
x, y, *rest = lst
x, y, rest

In [ ]:
lst = [1, 2]
[3,4, *lst, lst, 5,6]

In [ ]:
args = [1,2,3,4]
args_ = []
for x in args:
    args_.append(x**2)

[x**2 for x in args]

In [ ]:
args = [1,2]
def add(x,y):
    return x + y
add(1,2)
add(x=1,y=2)
add(args[0], args[1])
add(*args)

In [ ]:
arg_names = ['x', 'y']
arg_values = [1, 2]
# quero que vire isso: {'x': 1, 'y', 2}

dict(zip(arg_names, arg_values))

In [ ]:
#unir dois dicionarios
arg_names = ['x', 'y']
arg_values = [1, 2]

dic1 = dict(zip(arg_names, arg_values))
dic2 = default_context

{**dic1, **dic2} #Essa maneira não é muito eficiente, pois é muito custo dependendo do dic
# Também não altera o dicionario, apenas gera um novo


# Essa maneira, suja o contexto, pois joga as variaveis sempre para o escopo global
# dic2.update(dic1)
# dic2

cm = ChainMap(dic1, dic2)
# cm
